In [22]:
import sys
from scratch_save import get_job_data
sys.path.append("UF")
sys.path.append("..")
sys.path.append("../UF")
sys.path.append("../.Scratch/")
%load_ext autoreload
%autoreload 
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from qiskit_qec.circuits import ArcCircuit
from qiskit_qec.decoders import UnionFindDecoder
from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np

In [3]:
links = [[0, 1, 2], [2, 3, 4], [4, 5, 6], [6, 7, 8]]
T = 5 # number of syndrome measurement rounds
code = ArcCircuit(links, T, basis='zz',logical='1')
qc = code.circuit['zz']
qc.draw(fold=-1)

░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                        
      link_qubit_0: ──────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├────────────────────────
                          ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘┌─┐                     
      link_qubit_1: ──────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫─┤M├─────────────────────
                          ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐                  
      link_qubit_2: ──────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫──╫─┤M├──────────────────
                          ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║  ║ └╥┘┌─┐               
      link_qubit_3: ──────░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫──╫──╫─┤M├───────────────
                    ┌───┐ ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║  ║  ║ └╥┘┌─┐            
      code_qubit_0: ┤ X ├─░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫──╫──╫──╫─┤M├────────────
                    ├───┤ ░        │    │    │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   

In [4]:
counts = AerSimulator().run(qc).result().get_counts()
fake_counts = FakeSherbrooke().run(qc, shots=10000).result().get_counts()
fake_counts = sorted(fake_counts.items(), key=lambda x: x[1], reverse=True)
print(str(counts))
print(fake_counts)

{'11111 0000 0000 0000 0000 0000': 1024}
[('11111 0000 0000 0000 0000 0000', 5334), ('11011 0000 0000 0000 0000 0000', 647), ('11111 0000 0000 0000 0001 0000', 234), ('11111 0000 0000 0000 0000 0001', 225), ('11111 0000 0001 0000 0000 0000', 205), ('11111 0000 0000 0001 0000 0000', 195), ('11111 0001 0000 0000 0000 0000', 194), ('11111 0000 0100 0000 0000 0000', 160), ('11111 0000 0000 0000 0100 0000', 159), ('11111 0100 0000 0000 0000 0000', 153), ('11111 0000 0000 0000 0000 0100', 138), ('11111 0000 0000 0100 0000 0000', 123), ('11101 0000 0000 0000 0000 0000', 94), ('11111 0000 0010 0000 0000 0000', 77), ('11111 0000 0000 0000 0010 0000', 77), ('11111 0000 0000 0010 0000 0000', 76), ('11111 0010 0000 0000 0000 0000', 76), ('11111 0000 0000 0000 1000 0000', 75), ('11111 0000 0000 0000 0000 1000', 72), ('11111 0000 1000 0000 0000 0000', 71), ('01111 0000 0000 0000 0000 0000', 69), ('11111 0000 0000 0000 0000 0010', 68), ('11111 1000 0000 0000 0000 0000', 68), ('11111 0000 0000 1000 00

# Count order

In [5]:
insert_position = next((index for index, instr in enumerate(qc.data) if instr[0].name == 'barrier'), None) + 1
print("insert_position: ", insert_position)
print(qc.data)

insert_position:  6
[CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 2),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 3),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 4),), clbits=()), CircuitInstruction(operation=Instruction(name='barrier', num_qubits=9, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(4, 'link_qubit'), 0), Qubit(QuantumRegister(4, 'link_qubit'), 1), Q

In [6]:
new_qc = qc.copy()
new_qc.data.clear()
new_qc.data += qc.data[:insert_position]
new_qc.x(0)
new_qc.data += qc.data[insert_position:]
new_qc.draw(fold=-1)

░ ┌───┐     ┌───┐          ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                     ░ ┌───┐                    ┌───┐           ░ ┌─┐                        
      link_qubit_0: ──────░─┤ X ├─────┤ X ├──────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├─|0>─────────────────░─┤ X ├────────────────────┤ X ├───────────░─┤M├────────────────────────
                          ░ └───┘     └─┬─┘          └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░ └─┬─┘                    └─┬─┘┌───┐┌───┐ ░ └╥┘┌─┐                     
      link_qubit_1: ──────░─────────────┼──────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░───┼────────────────────────┼──┤ X ├┤ X ├─░──╫─┤M├─────────────────────
                          ░ ┌───┐       │       ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░   │  ┌───┐          ┌───┐  │  └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐                  
      link_qubit_2: ──────░─┤ X ├───────┼───────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫───╫───┤M├──|0>───────░───┼──┤ X ├──────────┤ X ├──┼────┼────┼───░──╫──╫─┤M├──────────────────
                          ░ └─┬─┘┌───┐  │  ┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░   │  └─┬─┘┌───┐┌───┐└─┬─┘  │    │    │   ░  ║  ║ └╥┘┌─┐               
      link_qubit_3: ──────░───┼──┤ X ├──┼──┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░───┼────┼──┤ X ├┤ X ├──┼────┼────┼────┼───░──╫──╫──╫─┤M├───────────────
                    ┌───┐ ░   │  └─┬─┘  │  └─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║   ║    ║   └╥┘       ░   │    │  └─┬─┘└─┬─┘  │    │    │    │   ░  ║  ║  ║ └╥┘┌─┐            
      code_qubit_0: ┤ X ├─░───┼────┼────■────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───■────┼────┼────┼────┼────┼────┼────┼───░──╫──╫──╫──╫─┤M├────────────
                    ├───┤ ░   │    │         │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   ░  ║   ║    ║    ║        ░        │    │    │    │    │    │    │   

In [7]:
counts = AerSimulator().run(new_qc).result().get_counts()
fake_counts = FakeSherbrooke().run(new_qc, shots=100).result().get_counts()
fake_counts = sorted(fake_counts.items(), key=lambda x: x[1], reverse=True)
print(str(counts))
print(fake_counts)

{'11111 0000 0000 0000 0000 0001': 1024}
[('11111 0000 0000 0000 0000 0001', 53), ('11111 0000 0000 0000 0001 0001', 7), ('11011 0000 0000 0000 0000 0001', 6), ('11111 0000 0000 0001 0000 0001', 3), ('11111 0000 0000 0000 1000 0001', 2), ('11111 0000 1000 0000 0000 0001', 2), ('11111 0001 0000 0000 0000 0001', 2), ('11111 0100 0000 0000 0000 0001', 2), ('11111 0000 0000 1000 0000 0001', 2), ('11111 0000 0001 0000 0000 0001', 2), ('10111 0000 0000 0000 0000 0001', 2), ('11111 0000 0000 0000 0100 0001', 1), ('11111 0101 0000 0000 0000 0001', 1), ('11011 0100 0000 0000 0000 0001', 1), ('11111 0000 0000 0000 0101 0001', 1), ('11111 0000 0000 0000 0010 0001', 1), ('11011 0100 0000 0000 0100 0101', 1), ('11111 0100 0000 0000 0000 0101', 1), ('11111 0001 1001 0010 0000 0101', 1), ('11111 0000 0100 0000 0000 0001', 1), ('11111 0000 0000 0000 0000 0011', 1), ('11111 0000 0010 0000 0000 0001', 1), ('01111 0000 0000 0010 0100 0001', 1), ('01111 0000 0010 0000 0000 0001', 1), ('11111 1000 0001 000

# IQ Data Order

In [8]:
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile
from probabilities_functions import get_counts

provider = IBMProvider()
backend = provider.get_backend('ibmq_jakarta')

links = [[0, 1, 2], [2, 3, 4]]
T = 3 # number of syndrome measurement rounds
code = ArcCircuit(links, T, basis='zz',logical='1')

qc = code.circuit['zz']

new_qc = qc.copy()
new_qc.data.clear()
new_qc.x(0)
new_qc.data += qc.data


qc = new_qc
qc.draw(fold=-1)


┌───┐ ░ ┌───┐┌───┐           ░ ┌─┐           ░ ┌───┐┌───┐           ░ ┌─┐           ░ ┌───┐┌───┐           ░ ┌─┐            
      link_qubit_0: ┤ X ├─░─┤ X ├┤ X ├───────────░─┤M├─|0>───────░─┤ X ├┤ X ├───────────░─┤M├─|0>───────░─┤ X ├┤ X ├───────────░─┤M├────────────
                    └───┘ ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐       ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐       ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘┌─┐         
      link_qubit_1: ──────░───┼────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>──░───┼────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>──░───┼────┼──┤ X ├┤ X ├─░──╫─┤M├─────────
                    ┌───┐ ░   │    │  └─┬─┘└─┬─┘ ░  ║  └╥┘       ░   │    │  └─┬─┘└─┬─┘ ░  ║  └╥┘       ░   │    │  └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐      
      code_qubit_0: ┤ X ├─░───■────┼────┼────┼───░──╫───╫────────░───■────┼────┼────┼───░──╫───╫────────░───■────┼────┼────┼───░──╫──╫─┤M├──────
                    ├───┤ ░        │    │    │   ░  ║   ║        ░        │    │    │   ░  ║   ║        ░        │    │    │   ░  ║  ║ └╥┘┌─┐   
      code_qubit_1: ┤ X ├─░────────■────■────┼───░──╫───╫────────░────────■────■────┼───░──╫───╫────────░────────■────■────┼───░──╫──╫──╫─┤M├───
                    ├───┤ ░                  │   ░  ║   ║        ░                  │   ░  ║   ║        ░                  │   ░  ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ┤ X ├─░──────────────────■───░──╫───╫────────░──────────────────■───░──╫───╫────────░──────────────────■───░──╫──╫──╫──╫─┤M├
                    └───┘ ░                      ░  ║   ║        ░                      ░  ║   ║        ░                      ░  ║  ║  ║  ║ └╥┘
round_0_link_bit: 2/════════════════════════════════╩═══╩══════════════════════════════════╬═══╬══════════════════════════════════╬══╬══╬══╬══╬═
                                                    0   1                                  ║   ║                                  ║  ║  ║  ║  ║ 
round_1_link_bit: 2/═══════════════════════════════════════════════════════════════════════╩═══╩══════════════════════════════════╬══╬══╬══╬══╬═
                                                                                           0   1                                  ║  ║  ║  ║  ║ 
round_2_link_bit: 2/══════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╬══╬══╬═
                                                                                                                                  0  1  ║  ║  ║ 
        code_bit: 3/════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩═
                                                                                                                                        0  1  2

In [9]:

initial_layout = [1, 5, 0, 3, 6] # for link_qubit and code_qubit [link_qubit, code_qubit]
transpiled_qc = transpile(qc, backend, initial_layout=initial_layout) #* virtual to physical: [0, 3, 5]  
job_id = backend.run(transpiled_qc, shots=1000, meas_level=1, meas_return='single').job_id()  # meas_level 1: IQ (0 is counts), meas_return: single for info from every shot (not 'avg')
job_id


'cms8ay30c0f00089ec80'

In [10]:
job = provider.retrieve_job(job_id)
job.status()

<JobStatus.RUNNING: 'job is actively running'>

In [11]:
memory = job.result().get_memory()
np.shape(memory) 
counts = get_counts(memory)
counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
counts

[('100000111', 236),
 ('100101110', 53),
 ('100001110', 47),
 ('100011101', 40),
 ('101111101', 38),
 ('100001111', 34),
 ('100000011', 25),
 ('110101110', 23),
 ('101010011', 23),
 ('100010011', 20),
 ('100000110', 16),
 ('101000111', 16),
 ('110101111', 16),
 ('110000111', 16),
 ('100110100', 15),
 ('100101111', 15),
 ('100100110', 14),
 ('000000111', 11),
 ('110100110', 11),
 ('100111101', 11),
 ('110001111', 11),
 ('110100111', 10),
 ('100011010', 9),
 ('100000101', 8),
 ('011111101', 8),
 ('110001110', 7),
 ('100101001', 7),
 ('100010100', 7),
 ('101010100', 6),
 ('001111101', 6),
 ('100100111', 6),
 ('101110101', 5),
 ('101001111', 5),
 ('100000000', 5),
 ('100001010', 4),
 ('000101110', 4),
 ('101101101', 4),
 ('000000110', 4),
 ('100101011', 4),
 ('101110100', 4),
 ('101001110', 4),
 ('101011101', 4),
 ('111111100', 3),
 ('101101110', 3),
 ('001010011', 3),
 ('100001001', 3),
 ('111111101', 3),
 ('110110100', 3),
 ('100111100', 3),
 ('110111101', 3),
 ('100111110', 3),
 ('10000

### ```=> IQ data is chronologicall but count data is inversed chronologicall```

In [12]:
import os
fiel = os.path.basename(globals()['__vsc_ipynb_file__'])
print(f"file: {fiel}")

file: Data_order_.ipynb


In [13]:
memory.tolist()
np.shape(memory.tolist())

(1000, 9)

In [21]:
from scratch_save import get_job_data

memory = get_job_data(job_id, provider, globals())


Job result has memory, getting memory
Data saved to /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/2023/10-Oct/20.10.23/16:02Data_order_-jakarta.json
